In [70]:
from __future__ import unicode_literals, print_function
import io
import glob
import os
import string 
import unicodedata
import random 
import time 

import torch
import torch.nn as nn
import torch.optim as optim

random.seed(0)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
all_letters = string.ascii_letters+" .,;'"
n_letters = len(all_letters)

In [10]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def read_file(file):
    f = open(file, encoding='utf-8')
    lines = f.readlines()
    return [unicode_to_ascii(i) for i in lines]
                 
data = {}
all_cats = []

files = glob.glob('data/nlp_data/ndata/names/*.txt')
for file in files:
    cat = file.split('/')[-1][:-4]
    lines = read_file(file)
    data[cat] = lines
    all_cats.append(cat)

n_cats = len(all_cats)

In [11]:
all_cats

['French',
 'Spanish',
 'Arabic',
 'English',
 'Czech',
 'Greek',
 'Dutch',
 'Irish',
 'Korean',
 'German',
 'Vietnamese',
 'Italian',
 'Polish',
 'Scottish',
 'Chinese',
 'Portuguese',
 'Japanese',
 'Russian']

In [149]:
letter_to_idx = {j: i for i,j in enumerate(all_letters)}

def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for i, letter in enumerate(line):
        tensor[i][0][letter_to_idx[letter]] = 1
    return tensor

print(line_to_tensor('Adam').size())
print(line_to_tensor("Adam"))

torch.Size([4, 1, 57])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

In [146]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size+hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size+hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), 1).to(device)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_cats)
rnn.to(device)
print(rnn)

RNN(
  (i2h): Linear(in_features=185, out_features=128, bias=True)
  (i2o): Linear(in_features=185, out_features=18, bias=True)
  (softmax): LogSoftmax()
)


In [76]:
sample = line_to_tensor("Adam")
print(sample.size())
letter = sample[0]
print(letter.size())
hidden = torch.zeros(1, n_hidden)

out, next_hidden = rnn(letter, hidden)
print(out)

torch.Size([4, 1, 57])
torch.Size([1, 57])
tensor([[ 0.0432,  0.0640, -0.0539, -0.0068,  0.0173,  0.0774,  0.0394, -0.0990,
         -0.0515, -0.0412,  0.1336,  0.0277,  0.0838,  0.0409, -0.0053, -0.1179,
         -0.0523,  0.0547]], grad_fn=<AddmmBackward>)


In [27]:
def cat_from_output(output):
    _, top = torch.max(output, dim = 1)
    idx = top.item()
    return all_cats[idx], idx

In [31]:
print(cat_from_output(out))

('Italian', 11)


In [32]:
def random_training():
    category = random.choice(all_cats)
    line = random.choice(data[category])
    
    label = torch.tensor([all_cats.index(category)], dtype=torch.long)
    inputs = line_to_tensor(line)
    
    return inputs, label

In [105]:
train_data = [random_training() for _ in range(10000)]
train_data[0]

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.,

In [144]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(rnn.parameters(), lr = .001)

In [147]:
def train(model, optimizer, epochs):
    start = time.time()

    for epoch in range(epochs):
        running_loss = 0.0
        running_correct = 0
        
        e_start = time.time()
        print(f'Epoch - {epoch+1}/{epochs}')
        print('-'*20)    
        
        
        for data in train_data:
            inputs, labels = data
            labels =  labels.to(device)
            hidden = model.init_hidden()

            optimizer.zero_grad()
            
            for i in range(inputs.size()[0]):
                letter = inputs[i][0].unsqueeze(0)
                output, hidden = model(letter, hidden)
                
            
            #test = nn.LogSoftmax()(output)
            #test = nn.NLLLoss()(output, labels)
            #print(test)
            _, pred = torch.max(output, dim = 1)
            loss = criterion(output, labels)

            #print(pred)
            
            #print(loss)
            #input()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_correct += torch.sum(pred == labels.data)

        epoch_loss = running_loss/len(train_data)
        acc = (running_correct.double()/len(train_data))*100
        e_end = time.time()-e_start
        print(f'loss - {epoch_loss:3.2f} acc - {acc:3.2f} , Time - {e_end:.2f}')
        
    end = time.time() - start
    print(f'Training time - {end:.2f}')
    return model

In [148]:
trained_model = train(rnn, optimizer, 5)

Epoch - 1/5
--------------------
loss - 2.89 acc - 10.15 , Time - 14.48
Epoch - 2/5
--------------------
loss - 2.89 acc - 10.15 , Time - 25.55
Epoch - 3/5
--------------------
loss - 2.89 acc - 10.15 , Time - 25.19
Epoch - 4/5
--------------------


KeyboardInterrupt: 